In [1]:
import requests
from bs4 import BeautifulSoup
import re
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import matplotlib.pyplot as plt
import warnings

In [2]:
import numpy as np
#import pandas as pd
#import re
import os
import codecs
#from sklearn import feature_extraction
import mpld3
import sys
import simplejson
import json
import pickle
import os

In [7]:
#Doc2Vec/Word2vec packages
import nltk
from nltk.tokenize import sent_tokenize
import gensim
from gensim import corpora, models, similarities
import logging
import os
import wikipedia
import random
from gensim.test.test_doc2vec import ConcatenatedDoc2Vec
from nltk.corpus import stopwords

In [4]:
warnings.filterwarnings('ignore')

###Loop through many wikipedia pages
Runs them through training model

Wikipedia API guide:
https://en.wikipedia.org/wiki/Special:ApiSandbox#action=query&prop=info&format=json&inprop=url&pageids=1000005

In [80]:
%%time
i = 44962         #i = 44962  (12-4-15 11:43 pm)
counter = 9103   #counter = 9103 (12-4-15 11:43 pm)
#good_id_list = [] #be sure only to initialize as empty list the first time
#id_list = []      #use new list if you want to preserve good_id_list in memory

while counter < 50000:   #number is limit of number of page ids to get
    scrape_worked = False
    #if i == 1000:
    #    print 'went on too long'
    #    break
    page_id = 1000000 + i
    
    #try to scrape webpage.  Ignore if bad page error is returned from wikipedia
    try:
        r = requests.get('https://en.wikipedia.org/w/api.php?action=query&prop=info\
                &format=json&inprop=url&pageids=' + str(page_id))
        rtext = r.text
        jsontext = json.loads(rtext)
        if jsontext['query']['pages'][str(page_id)]['ns'] == 0:
            scrape_worked = True
    except:
        i += 1    
    
    if scrape_worked == True:
        article = requests.get('https://en.wikipedia.org/?curid=' + str(page_id))
        soup = BeautifulSoup(article.text, 'html.parser')
        paragraphs = soup.findAll('p')
        if len(paragraphs) >= 10:
            good_id_list.append(page_id)
            #id_list.append(page_id)    #for preservation of good_id_list
            corpus = page_read(soup)
            
            #write each wikipedia article to its own file
            with open("/users/markregalla/desktop/metis/Project4/WikiArticles/page_" 
                      + str(page_id) + ".txt", "w") as myfile:
                myfile.write(corpus.encode("UTF-8"))
            myfile.close
            counter += 1
    i += 1
    
    #prints iterators as a status
    sys.stdout.write("\r" + 'Attempts: ' + str(i) + '  Sucesses: ' + str(counter))   

print '\n' + str(len(good_id_list))
#print '\n' + str(len(id_list))

Attempts: 259987  Sucesses: 50000
50000
CPU times: user 3h 23min 41s, sys: 8min 50s, total: 3h 32min 32s
Wall time: 14h 10min 12s


In [31]:
#Get article names from saved files in directory
#good_id_list = []
#for article in os.listdir('/users/markregalla/desktop/metis/Project4/WikiArticles'):
#    good_id_list.append(article[5:12])
good_id_list[0]    

'1000003'

##Clean data for modelling

In [36]:
#%%time

TaggedDocument = gensim.models.doc2vec.TaggedDocument
u = 0
model_list = []
paralist = []

#for loop for opening each file in directory for training
for article in os.listdir('/users/markregalla/desktop/metis/Project4/WikiArticles'):
    
    #Open saved total_corpus file for training
    with open("/users/markregalla/desktop/metis/Project4/WikiArticles/" 
              + article, "r") as myfile:
        para = myfile.read()
    myfile.close
    
    #Clean
    para = [para]
    para = cleanText(para)
    #paralist.append(para)
    
    #Save cleaned file in same place
    with open("/users/markregalla/desktop/metis/Project4/WikiArticles/" 
              + article, "w") as myfile:
        myfile.write(para)
    myfile.close
    
    sys.stdout.write("\r" + str(u))   #prints iterator as a status
    u += 1
    if u == 5:
        break

4

In [44]:
'that' in stop

True

In [ ]:
%%time
paralist = labelizeReviews(paralist, 'Testing')

##Functions for cleaning text

In [4]:
def page_read(soup):
    #thetext = []
    thetext = ''
    paragraphs = soup.findAll('p') #comment out only when saving wiki articles to disk
    for p in paragraphs:
        #thetext.append(p.text)    #make a list of sentences
        thetext = thetext + p.text + '\n' #make a list with 1 element being the whole document
    #thetext = [thetext]
    #theindex = 0
    #wikidef = paragraphs[theindex].text
    #print thetext
    #print type(thetext)
    return thetext

In [35]:
def cleanText(corpus):
    corpus = [z.lower() for z in corpus]
    corpus = re.sub('\n', ' ', corpus[0])
    #corpus = re.sub('\\[.*?\\]',' ', corpus)
    corpus = " ".join(re.split("[^a-zA-Z.]*", corpus))
    stop = stopwords.words('english')
    for word in corpus:
        if word in stop:
            word.replace(word, '')
    
    #treat punctuation as individual words
    #punctuation = """.,?!:;(){}[]"""
    #for c in punctuation:
    #    corpus = [z.replace(c, ' %s '%c) for z in corpus]
    #corpus = [z.split() for z in corpus]
    return corpus

In [175]:
def labelizeReviews(reviews, label_type):
    labelized = []
    for i,v in enumerate(reviews):
        #label = '%s_%s'%(label_type,i)    #iterate sentence labels
        label = '%s_%s'%(label_type,1)     #uniform sentence labels
        labelized.append(TaggedDocument(v, [label]))
    return labelized
    #yield LabeledSentence(words=line.split(), labels=['SENT_%s' % uid])
        #yield gensim.models.doc2vec.LabeledSentence(words=line.split(), tags=['SENT_%s' % uid])

In [ ]:
class LabeledLineSentence(object):
   def __init__(self, filename):
       self.filename = filename
   def __iter__(self):
       for uid, line in enumerate(open(filename)):
           yield gensim.models.doc2vec.TaggedDocument(words=line.split(), tags=['SENT_%s' % uid])

In [20]:
#Julia's version
stop = stopwords.words()

class MySentences2(object):
    def __init__(self, dirname):
           self.dirname = dirname

    def __iter__(self):
        counter = 0
        for fname in os.listdir(self.dirname):
            for uid, line in enumerate(open(os.path.join(self.dirname, fname))):
                print sys.stdout.write("\r" + fname)
                yield gensim.models.doc2vec.TaggedDocument(words=line.split(), tags=['SENT_%s' % uid])
            counter += 1
            if counter == 20:
                break

In [ ]:
#my version with regex to remove wikipedia reference numbers

stop = stopwords.words('english')

class MySentences2(object):
    def __init__(self, dirname):
           self.dirname = dirname

    def __iter__(self):
        counter = 0
        for fname in os.listdir(self.dirname):
            print sys.stdout.write("\r" + fname)
            for uid, line in enumerate(open(os.path.join(self.dirname, fname))):
                word = line.lower().split()
                word = re.sub('\\[.*?\\]',' ', word)
                if word not in stop:
                    yield gensim.models.doc2vec.LabeledSentence(word, tags=['SENT_%s' % uid])
            counter += 1
            if counter == 20:
                break

##Modelling

In [21]:
mysentences = MySentences2('/users/markregalla/desktop/metis/Project4/WikiArticles')

In [22]:
wiki_model = gensim.models.Doc2Vec(mysentences, size=100, window=8, min_count=5, workers=4)

page_1000007.txtNone
page_1000007.txtNone
page_1000007.txtNone
page_1000007.txtNone
page_1000007.txtNone
page_1000007.txtNone
page_1000007.txtNone
page_1000007.txtNone
page_1000007.txtNone
page_1000007.txtNone
page_1000007.txtNone
page_1000007.txtNone
page_1000007.txtNone
page_1000007.txtNone
page_1000007.txtNone
page_1000007.txtNone
page_1000021.txtNone
page_1000021.txtNone
page_1000021.txtNone
page_1000021.txtNone
page_1000021.txtNone
page_1000021.txtNone
page_1000021.txtNone
page_1000021.txtNone
page_1000021.txtNone
page_1000021.txtNone
page_1000021.txtNone
page_1000021.txtNone
page_1000021.txtNone
page_1000021.txtNone
page_1000021.txtNone
page_1000021.txtNone
page_1000021.txtNone
page_1000021.txtNone
page_1000021.txtNone
page_1000021.txtNone
page_1000021.txtNone
page_1000021.txtNone
page_1000021.txtNone
page_1000021.txtNone
page_1000021.txtNone
page_1000021.txtNone
page_1000021.txtNone
page_1000021.txtNone
page_1000021.txtNone
page_1000021.txtNone
page_1000021.txtNone
page_1000021.

In [23]:
wiki_model.vocab

{'Himmerszeck)': <gensim.models.word2vec.Vocab at 0x1114c83d0>,
 'Ehr': <gensim.models.word2vec.Vocab at 0x1114c8410>,
 'Many': <gensim.models.word2vec.Vocab at 0x10c652950>,
 'limited': <gensim.models.word2vec.Vocab at 0x1029e6590>,
 'assembles': <gensim.models.word2vec.Vocab at 0x1029e65d0>,
 'Syldavia': <gensim.models.word2vec.Vocab at 0x1029e6610>,
 'destroyed': <gensim.models.word2vec.Vocab at 0x1029ecc50>,
 'four': <gensim.models.word2vec.Vocab at 0x10c542c50>,
 'Arabex,': <gensim.models.word2vec.Vocab at 0x102a07e90>,
 'ocean.': <gensim.models.word2vec.Vocab at 0x10c526750>,
 'consists': <gensim.models.word2vec.Vocab at 0x10c526990>,
 'captain': <gensim.models.word2vec.Vocab at 0x1114c84d0>,
 'saved': <gensim.models.word2vec.Vocab at 0x102a07ed0>,
 '(which': <gensim.models.word2vec.Vocab at 0x1114c8510>,
 '(Colonel': <gensim.models.word2vec.Vocab at 0x1114c8550>,
 'disobeying': <gensim.models.word2vec.Vocab at 0x10c526710>,
 'up,': <gensim.models.word2vec.Vocab at 0x1114c85d0>,


In [ ]:
%%time
wiki_model = gensim.models.Doc2Vec(paralist, size=100, window=8, min_count=5, workers=4)

#print all_model.docvecs['Testing_0']

wiki_model.vocab

In [ ]:
concat_model = ConcatenatedDoc2Vec([x for x in model_list])
print concat_model.docvecs['Testing_0']

In [90]:
doc1=["This is a sentence","This is another sentence"]
documents=[doc.strip().split(" ") for doc in doc1 ]
model = gensim.models.Doc2Vec(documents, size = 100, window = 300, min_count = 10, workers=4)

AttributeError: 'list' object has no attribute 'words'

In [93]:
documents = ["Human machine interface for lab abc computer applications",
            "A survey of user opinion of computer system response time",
            "The EPS user interface management system",
            "System and human system engineering testing of EPS",
             "Relation of user perceived response time to error measurement",
             "The generation of random binary unordered trees",
             "The intersection graph of paths in trees",
             "Graph minors IV Widths of trees and well quasi ordering",
             "Graph minors A survey"]
stoplist = set('for a of the and to in'.split())
texts = [[word for word in document.lower().split() if word not in stoplist]
        for document in documents]
model = gensim.models.Doc2Vec(texts, size = 100, window = 300, min_count = 10, workers=4)

AttributeError: 'list' object has no attribute 'words'

In [ ]:
#Sample URLS for testing content
url = ['https://en.wikipedia.org/wiki/Myocardial_infarction',
      'https://en.wikipedia.org/wiki/Blood_flow',
      'https://en.wikipedia.org/wiki/Circulatory_system#Human_cardiovascular_system']

##Pickling

In [ ]:
#dump list of good id nos into pickle file
with open('paralist.pkl', 'w') as picklefile:
    #pickle.dump(good_id_list, picklefile)
    pickle.dump(paralist, picklefile)

In [ ]:
#retrieve list of good id nos
with open('good_id_list_1.pkl', 'r') as picklefile:
    good_id_list = pickle.load(picklefile)